# Setup

In [1]:
# pip install ir_datasets
import ir_datasets
dataset = ir_datasets.load("clinicaltrials/2021")

In [3]:
# for doc in dataset.docs_iter():
#     doc # namedtuple<doc_id, title, condition, summary, detailed_description, eligibility>

In [5]:
# !pip install elasticsearch==7.9.1
# !pip install elasticsearch==7.17.3

In [7]:
from elasticsearch import Elasticsearch
es = Elasticsearch(HOST='http://localhost', PORT='9200')

In [8]:
mapping = {
    "settings":
    {
        "index":
        {
            "number_of_shards":1,
            "number_of_replicas":1
        }
    },
    "mappings":
    {
        "properties":
        {
            "content":
            {
                "type":"text",
                "fielddata":True,
                "term_vector":"with_positions_offsets_payloads",
                "store":True,
                "analyzer":"whitespace"
            }
        }
    }
}


In [9]:
es.indices.create(index='ir3', body=mapping)

RequestError: RequestError(400, 'resource_already_exists_exception', 'index [ir3/8QDV0gunS_6tkQp4-kqmfQ] already exists')

In [10]:
es.indices.exists(index='ir3')

True